# To-Do:
- Get per subject AUROC scores and variance for each method.
- Check if ensemble result analysis is handled properly: Shape of ensemble prediction set is (9, 10, 576, 4). This is the only case of such a shape and I don't know if it was handled well by the AUROC and accuracy calculating functions
- Fix Flipout predictions: Issue is that I use X_test shape while predicting for lockbox which has a different shape. This affects the KL-weight parameter for flipout. Also, I don't know if this value is only fixed for the training set.
- Tidy up code: Remove unneccesary files, make functions as general as possible, make a lot of documentation.

In [1]:
# Set notebook to use only one GPU
%env CUDA_VISIBLE_DEVICES=1     
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=1


In [2]:
from result_analysis_functions import *

Using TensorFlow backend


Keras Uncertainty will use standalone Keras backend

# Accuracy and average uncertainty:
Data shape of preds and lockbox: (9, 50, 576, 4), 50 forward passes. 

'''
preds and lockbox shape: (9, 50, 576, 4).
Axis -3 (50) should be collapsed only in the 
following cases:
    - accuracy
    - std. dev.
Collapsing of the axis for uncertainty is in
models_bachelors.py
'''

### Accuracy of each method

In [ ]:
'''
CLEANUP IN PROGRESS
'''

acc_methods = {'mcdropconnect': {'test': [], 'lockbox': []},
               'mcdropout': {'test': [], 'lockbox': []},
               'flipout': {'test': [], 'lockbox': []},
               'ensemble': {'test': [], 'lockbox': []},
               'duq': {'test': [], 'lockbox': []},
               'standard': {'test': [], 'lockbox': []},
               'standard_dropconnect': {'test': [], 'lockbox': []}
               }
NUM = 50  # Number of prediction sets

for method, dicts in acc_methods.items():
    preds = load_predictions(method)
# Load each prediction set and compute avg accuracy for each set, adding them to list
for n in range(NUM):
    methods = load_dict_from_hdf5(f'predictions/predictions_duq.h5')
    for name, method in methods.items():
        isStandard = True
        # print(f'y_true: {method["test"]["labels"].argmax(axis=-1).shape}, y_pred: {method["test"]["preds"].argmax(axis=-1).shape}')
        test_accs = get_accuracies(method["test"], isStandard)   # Get test set accuracies
        lock_accs = get_accuracies(method["lockbox"], isStandard)
        if name == 'mcdropconnect':
            acc_mcdropconnect['test'].append(test_accs)
            acc_mcdropconnect['lockbox'].append(lock_accs)
        elif name == 'mcdropout':
            acc_mcdropout['test'].append(test_accs)
            acc_mcdropout['lockbox'].append(lock_accs)
        elif name == 'standard':
            acc_std['test'].append(test_accs)
            acc_std['lockbox'].append(lock_accs)
        elif name == 'standard_dropconnect':
            acc_std_mcdropconnect['test'].append(test_accs)
            acc_std_mcdropconnect['lockbox'].append(lock_accs)
        elif name == 'flipout':
            acc_flipout['test'].append(test_accs)
            acc_flipout['lockbox'].append(lock_accs)
        elif name == 'duq':
            acc_duq['test'].append(test_accs)
            acc_duq['lockbox'].append(lock_accs)
        else:
            acc_ensemble_dropout['test'].append(test_accs)
            acc_ensemble_dropout['lockbox'].append(lock_accs)

# Only for UQ:
for name, key in acc_mcdropconnect.items():
    key = np.array(key).mean(axis=0)

for name, key in acc_mcdropout.items():
    key = np.array(key).mean(axis=0)

for name, key in acc_ensemble_dropout.items():
    key = np.array(key).mean(axis=0)

for name, key in acc_flipout.items():
    key = np.array(key).mean(axis=0)



In [4]:
from numpy import round

r = 5
# print(round(np.std(acc_mcdropconnect["test"]), 5))
# print('mcdropout')
# print(acc_mcdropout)
# print(f'test set avg acc: {round(np.mean(acc_mcdropout["test"]), r) * 100} +/- {round(np.std(acc_mcdropout["test"]), r) * 100}')
# print(f'lockbox set avg acc: {round(np.mean(acc_mcdropout["lockbox"]), r) * 100} +/- {round(np.std(acc_mcdropout["lockbox"]), r) * 100}\n')

# print('mcdropconnect')
# print(f'test set avg acc: {round(np.mean(acc_mcdropconnect["test"]), r) * 100} +/- {round(np.std(acc_mcdropconnect["test"]), r) * 100}')
# print(f'lockbox set avg acc: {round(np.mean(acc_mcdropconnect["lockbox"]), r) * 100} +/- {round(np.std(acc_mcdropconnect["lockbox"]), r) * 100}\n')

# print('standard_dropout')
# print(f'test set avg acc: {round(np.mean(acc_std["test"]), r) * 100} +/- {round(np.std(acc_std["test"]), r) * 100}')
# print(f'lockbox set avg acc: {round(np.mean(acc_std["lockbox"]), r) * 100} +/- {round(np.std(acc_std["lockbox"]), r) * 100}\n')


# print('standard_dropconnect')
# print(f'test set avg acc: {round(np.mean(acc_std_mcdropconnect["test"]), r) * 100} +/- {round(np.std(acc_std_mcdropconnect["test"]), r) * 100}')
# print(f'lockbox set avg acc: {round(np.mean(acc_std_mcdropconnect["lockbox"]), r) * 100} +/- {round(np.std(acc_std_mcdropconnect["lockbox"]), r) * 100}\n')


# print('ensemble_dropout')
# print(f'test set avg acc: {round(np.mean(acc_ensemble_dropout["test"]), r) * 100} +/- {round(np.std(acc_ensemble_dropout["test"]), r) * 100}')
# print(f'lockbox set avg acc: {round(np.mean(acc_ensemble_dropout["lockbox"]), r) * 100} +/- {round(np.std(acc_ensemble_dropout["lockbox"]), r) * 100}\n')


print('duq')
print(f'test set avg acc: {round(np.mean(acc_duq["test"]), r) * 100} +/- {round(np.std(acc_duq["test"]), r) * 100}')
print(f'lockbox set avg acc: {round(np.mean(acc_duq["lockbox"]), r) * 100} +/- {round(np.std(acc_duq["lockbox"]), r) * 100}\n')

# print('flipout')
# print(f'test set avg acc: {round(np.mean(acc_flipout["test"]), r) * 100} +/- {round(np.std(acc_flipout["test"]), r) * 100}')
# print(f'lockbox set avg acc: {round(np.mean(acc_flipout["lockbox"]), r) * 100} +/- {round(np.std(acc_flipout["lockbox"]), r) * 100}\n')

duq
test set avg acc: 55.421 +/- 9.163
lockbox set avg acc: 70.468 +/- 2.929



# ROC plots


In [ ]:
from sklearn.metrics import roc_auc_score, auc
import matplotlib.patches as mpatches
from numpy import round

'''
Gets a numpy array down to a 2D array
'''
def get_in_shape(data):
    # If data has shape of 2 elements or less, assume it's already in shape
    if len(data.shape) < 3:
        return data
    else:
        while len(data.shape) > 2:
            data = np.vstack(data)
        return data


'''
data can be whatever shape.
manual roc plot creation reqs:
    y_pred: (9*50, 50, 576, 4)
'''
def make_roc_plot(y_true, y_pred, isStandard, unc_method):
    '''
    y_pred can be either of shape (50, 9, 50, 576, 4) or  (9, 50, 576, 4). We need it in shape (X, 4).
    y_true can be either of shape (50, 9, 576, 4) or (9, 576, 4).
    So apply same algorithm to get these sets into the shape (X, 4)
    '''
    thresholds = np.arange(0, 1.001, 0.001)
    # print(f'y_true: {y_true.shape}, y_pred: {y_pred.shape}')
    tpr = []
    fpr = []
    unc = get_uncertainty(y_pred, unc_method).flatten()
    y_true = get_in_shape(y_true)
    y_pred = get_in_shape(y_pred) if isStandard else get_in_shape(y_pred.mean(axis=-3))
    
    # print(f'y_true: {y_true.shape}, y_pred: {y_pred.shape}, certains: {unc.shape}')
    for t in thresholds:
        '''
        Order is reversed because FPR and TPR are reversed for some reason
        '''
        certains = (t < unc)           # Certain when uncertainty is below threshold
        uncertains = (t > unc)
        # Calculate TPR and FPR
        tp = sum(y_pred.argmax(axis=1)[uncertains] != y_true.argmax(axis=1)[uncertains])     # N. preds uncertain predictions that are incorrect
        fn = sum(y_pred.argmax(axis=1)[certains] != y_true.argmax(axis=1)[certains])      # Prediction that's certain and incorrect
        fp = sum(y_pred.argmax(axis=1)[uncertains] == y_true.argmax(axis=1)[uncertains])   # prediction that's uncertain and correct
        tn = sum(y_pred.argmax(axis=1)[certains] == y_true.argmax(axis=1)[certains]) # Prediction that's certain and correct
        # print(f'tp: {tp} fn: {fn} fp: {fp} tn: {tn}')
        fpr.append(fp / (fp + tn))
        tpr.append(tp / (tp + fn))
    # fig1, ax1 = plt.subplots()
    # hist_correct, bins_correct, _ = ax1.hist(auc, bins=10, density=False, alpha=0.5, label='Correct')
    # fig1.show()
    return np.array(tpr), np.array(fpr)


'''
I calculate AUROC and plot ROC separately because I want to get
mean AUROC of all 50 prediction sets along with their variance.
Then I plot ROC with all 50 prediction sets.
'''
def roc_plot_and_auroc(method, key, unc_method):
    y_pred = []
    y_true = []
    aucs_lst = []
    # isStandard = True if 'standard' in method else False
    isStandard = False
    # num_predictions = 50 if not isStandard else 1
    num_predictions = 1
    # creation of set of 50 predictions, as well as AUROC score calculation
    for n in range(num_predictions):
        # methods = load_predictions(n, 'duq')
        methods = load_dict_from_hdf5(f'predictions/predictions_ensemble_dropout.h5')
        data = methods[method][key]
        tpr, fpr = make_roc_plot(data['labels'], data['preds'], isStandard, unc_method)
        # print(f'y_true shape: {y_true_roc.shape} y_pred: {y_pred_roc.shape}')
        auroc_score = auc(tpr, fpr)
        aucs_lst.append(auroc_score)
        y_pred.append(data['preds'])
        y_true.append(data['labels'])

    tpr, fpr = make_roc_plot(np.vstack(y_true), np.vstack(y_pred), isStandard, unc_method)

    return tpr, fpr, aucs_lst


# aucs_test = {'predictive-entropy':
#           {'mcdropconnect':[], 'mcdropout':[], 'standard':[], 'standard_dropconnect':[]},
#           'shannon-entropy':
#           {'mcdropconnect':[], 'mcdropout':[], 'standard':[], 'standard_dropconnect':[]},
#           'mutual-information':
#           {'mcdropconnect':[], 'mcdropout':[], 'standard':[], 'standard_dropconnect':[]}
#         }
# aucs_test = {'predictive-entropy': {'mcdropconnect':[], 'mcdropout':[], 'standard':[], 'standard_dropconnect':[]}}
# methods = {'mcdropconnect':'C0', 'mcdropout':'C1', 'standard':'C4', 'standard_dropconnect':'C9'}
# aucs_test = {'predictive-entropy': {'ensemble_dropout': []}}
# aucs_test = {'predictive-entropy': {'duq': []}}
aucs_test = {'predictive-entropy': {'ensemble_dropout': []}, 'shannon-entropy': {'ensemble_dropout': []}, 'mutual-information': {'ensemble_dropout': []}}
# fig1, ax1 = plt.subplots(2, squeeze=False, figsize=(10, 20))
# ax1 = ax1.flatten()
key = "test"
print(key)
for unc_name, methods_dict in aucs_test.items():
    print(unc_name)
    for method, auc_lst in methods_dict.items():
        tpr, fpr, ret_aucs = roc_plot_and_auroc(method, key, unc_name)
        print(f'{key} AUC: {round(np.mean(ret_aucs), r) * 100} +/- {round(np.std(ret_aucs), r) * 100}')
        # Normalizes AREA UNDER CURVE to sum up to 1. y-axis values are meaningless.
        print(method)
        r = 6
        # ax1[0].plot(tpr, fpr, color=methods[method])

key = "lockbox"
print(key)
for unc_name, methods_dict in aucs_test.items():
    print(unc_name)
    for method, auc_lst in methods_dict.items():
        r = 6
        tpr, fpr, ret_aucs = roc_plot_and_auroc(method, key, unc_name)
        print(f'{key} AUC: {round(np.mean(ret_aucs), r) * 100} +/- {round(np.std(ret_aucs), r) * 100}')
        # Normalizes AREA UNDER CURVE to sum up to 1. y-axis values are meaningless.
        print(method)
        # ax1[1].plot(tpr, fpr, color=methods[method])

# red_patch = mpatches.Patch(color=methods['mcdropconnect'], label='MC-DropConnect')
# green_patch = mpatches.Patch(color=methods['mcdropout'], label='MC-Dropout')
# blue_patch = mpatches.Patch(color=methods['standard'], label='Standard Dropout')
# yellow_patch = mpatches.Patch(color=methods['standard_dropconnect'], label='Standard Dropconnect')

# ax1[0].legend(handles=[red_patch, green_patch, blue_patch, yellow_patch], fontsize=20)
# ax1[0].set_title(f'Out-of-population ROC', fontsize=35)
# ax1[0].set_xlabel("FPR", fontsize=30)
# ax1[0].set_ylabel("TPR", fontsize=30)

# ax1[1].legend(handles=[red_patch, green_patch, blue_patch, yellow_patch], fontsize=20)
# ax1[1].set_title(f'Within-population ROC', fontsize=35)
# ax1[1].set_xlabel("FPR", fontsize=30)
# ax1[1].set_ylabel("TPR", fontsize=30)

# # plt.rcParams.update({'font.size': 22})
# # fig1.savefig(f'roc_plot_pred_entropy_w_10_h_20.pdf')
# ax1[0].tick_params(axis='x', labelsize=20)
# ax1[0].tick_params(axis='y', labelsize=20)
# ax1[1].tick_params(axis='x', labelsize=20)
# ax1[1].tick_params(axis='y', labelsize=20)

# fig1.show()




In [ ]:
fig1.savefig(f'roc_plot_pred_entropy_w_10_h_20_large.pdf')


To see if there is a significant difference between the AUROCs of two methods with a certain metric on one set, need to construct and save a MATLAB struct with two elements:

- spsizes: 2 x 1 vector with samples sizes for X and Y, uncertainties and targets
- ratings: K x N matrix. K is uncertainties/targets of each model, it is the row. N is the sum of len(X) and len(Y) and first len(X) elements are uncertainties and last len(Y) elements are targets

Then I gotta save this data as a MATLAB struct while in python somehow

Things I want to compare for test and lockbox:
- MC-Dropout: mutual information and shannon entropy
- MC-DropConnect: mutual information and shannon entropy

So 2 separate files to save, one for test and the other for lockbox

y_true/ground truth labels will be 0, 1. And will be 1 when y_true != y_pred and 0 when it isn't. So basically, 1 for all incorrects (which should be uncertains) and 0 for all corrects (which should be certains).

# Per subject scores

In [ ]:
'''
Per-subject uncertainties and AUROC
This is exactly what I need:
    - Per subject AUROC. This can only be done with array of shape (9, 576).
        - start w/ (50, 9, 50, 576, 4) for a method.
        - Mean axis=0 -> (9, 50, 576, 4)
        - Get uncertainties -> (9, 576) -> Mean axis -1=Avg. uncertainties -> (9, 1)
        - For each subject in axis 0, calculate AUROC to get final array of (9, 1)
    - Array of shape (9, 1) for uncertanties
        - Get uncertainties -> (9, 576) -> Mean axis -1=Avg. uncertainties -> (9, 1)
'''

from sklearn.metrics import auc
import matplotlib.patches as mpatches
import scipy.stats as stats
from numpy import round

def load_all_predictions():
    predictions = {'mcdropout': 
            {'test': {'preds':[], 'labels':[]}, 
                'lockbox':{'preds':[], 'labels':[]}},
                'mcdropconnect': 
            {'test': {'preds':[], 'labels':[]}, 
                'lockbox':{'preds':[], 'labels':[]}},
                'standard': 
            {'test': {'preds':[], 'labels':[]}, 
                'lockbox':{'preds':[], 'labels':[]}},
                'standard_dropconnect': 
            {'test': {'preds':[], 'labels':[]}, 
                'lockbox':{'preds':[], 'labels':[]}}
            }
    # Load all UQ method predictions
    N = 50
    for n in range(N):
        dataset = load_predictions(n, False)
        # No need to append the labels because they're the same each time.
        # So only need to append them once.
        predictions['mcdropout']['test']['preds'].append(dataset['mcdropout']['test']['preds'])
        predictions['mcdropout']['lockbox']['preds'].append(dataset['mcdropout']['lockbox']['preds'])
        predictions['mcdropconnect']['test']['preds'].append(dataset['mcdropconnect']['test']['preds'])
        predictions['mcdropconnect']['lockbox']['preds'].append(dataset['mcdropconnect']['lockbox']['preds'])

    # Load all Standard method predictions
    dataset = load_predictions(1, True)
    predictions['standard']['test']['preds'].append(dataset['standard']['test']['preds'])
    predictions['standard']['lockbox']['preds'].append(dataset['standard']['lockbox']['preds'])
    predictions['standard_dropconnect']['test']['preds'].append(dataset['standard_dropconnect']['test']['preds'])
    predictions['standard_dropconnect']['lockbox']['preds'].append(dataset['standard_dropconnect']['lockbox']['preds'])

    # Load all the labels once into their respective method dicts
    for name, method in predictions.items():
        # standard labels are same as UQ method labels.
        method['test']['labels'] = dataset['standard']['test']['labels']
        method['lockbox']['labels'] = dataset['standard']['lockbox']['labels']
        # convert relevant lists to numpy arrays
        method['test']['preds'] = np.array(method['test']['preds'])
        method['lockbox']['preds'] = np.array(method['lockbox']['preds'])

    return predictions


'''
data can be whatever shape.
manual roc plot creation reqs:
    y_pred: (50, 576, 4)
'''
def get_fpr_tpr(y_true, y_pred, unc, isStandard):
    '''
    y_pred can be either of shape (50, 9, 50, 576, 4) or  (9, 50, 576, 4). We need it in shape (X, 4).
    y_true can be either of shape (50, 9, 576, 4) or (9, 576, 4).
    So apply same algorithm to get these sets into the shape (X, 4)
    '''
    thresholds = np.arange(0, 1.001, 0.001)
    tpr = []
    fpr = []
    y_pred = y_pred if isStandard else y_pred.mean(axis=-3)
    
    # print(f'y_true: {y_true.shape}, y_pred: {y_pred.shape}, certains: {unc.shape}')
    for t in thresholds:
        '''
        Order is reversed because FPR and TPR are reversed for some reason
        '''
        certains = (t < unc)           # Certain when uncertainty is below threshold
        uncertains = (t > unc)
        # Calculate TPR and FPR
        tp = sum(y_pred.argmax(axis=1)[uncertains] != y_true.argmax(axis=1)[uncertains])     # N. preds uncertain predictions that are incorrect
        fn = sum(y_pred.argmax(axis=1)[certains] != y_true.argmax(axis=1)[certains])      # Prediction that's certain and incorrect
        fp = sum(y_pred.argmax(axis=1)[uncertains] == y_true.argmax(axis=1)[uncertains])   # prediction that's uncertain and correct
        tn = sum(y_pred.argmax(axis=1)[certains] == y_true.argmax(axis=1)[certains]) # Prediction that's certain and correct
        # print(f'tp: {tp} fn: {fn} fp: {fp} tn: {tn}')
        fpr.append(fp / (fp + tn))
        tpr.append(tp / (tp + fn))
    return np.array(tpr), np.array(fpr)

def get_auroc(y_true, y_pred, unc, isStandard):
    tpr, fpr = get_fpr_tpr(y_true, y_pred, unc, isStandard)
    return auc(tpr, fpr)

def per_subject_metrics(data, isStandard):
    key_set = data[key]        # Whether lockbox or preds of the method
    y_true = key_set['labels']
    y_preds = key_set['preds'].mean(axis=0)     # CHANGE: FIRST AXIS IS NOT ALWAYS 50! FOR METHODS WITHOUT 50 SETS IT'S SIMPLY 9!
    unc = predictive_uncertainty(y_preds, 'predictive-entropy')
    per_subject_aucs = []
    # print(y_preds.shape)
    for subject_id in range(y_preds.shape[0]):
        per_subject_aucs.append(get_auroc(y_true[subject_id], y_preds[subject_id], unc[subject_id], isStandard))

    
    return np.array(per_subject_aucs), unc.mean(axis=1)

    


'''
data: (50, 9, 50, 576, 4)
method: 'mcdropconnect'/'mcdropout'/'standard'/'standard_dropconnect'
key: 'test'/'lockbox'
'''
def do_everything(data, method, key):
    # data shape for UQ preds: (50, 9, 50, 576, 4)
    isStandard = True if 'standard' in method else False
    aurocs, uncertainties = per_subject_metrics(data, isStandard)
    return aurocs, uncertainties


aucs = {'mcdropconnect':[], 'mcdropout':[], 'standard':[], 'standard_dropconnect':[]}
uncertainties = {'mcdropconnect':[], 'mcdropout':[], 'std':[], 'standard_dropconnect':[]}
methods = {'mcdropconnect':'red', 'mcdropout':'green', 'standard':'blue', 'standard_dropconnect':'yellow'}
fig1, ax1 = plt.subplots()
key = "test"
data = load_all_predictions()       # all prediction sets across all methods into a single object
for method, values in data.items():
    aurocs, uncs = do_everything(values, method, key)
    # Normalizes AREA UNDER CURVE to sum up to 1. y-axis values are meaningless.
    print(method)
    r = 6
    aucs[method] = aurocs
    uncertainties[method] = uncs
    # print(f'{key} set avg AUROC: {np.mean(aucs)} +/- {np.std(aucs)}')


In [ ]:
for method, values in aucs.items():
    print(f'\n\n{method}\n aucs:{values}\n uncs: {uncertainties[method]}')